In [1]:
#协程
def consumer():
    r = ''
    while True:
        n = yield r
        if not n:
            return
        print('[CONSUMER] Consuming %s...' % n)
        r = ' 200 OK'
def produce(c):
    c.send(None)
    n = 0
    while n < 5:
        n = n + 1
        print("[PRODUCER] Producing %s..." % n)
        r = c.send(n)
        print("[PRODUCER] Consumer return: %s" % r)
    c.close()

c = consumer()
produce(c)

[PRODUCER] Producing 1...
[CONSUMER] Consuming 1...
[PRODUCER] Consumer return:  200 OK
[PRODUCER] Producing 2...
[CONSUMER] Consuming 2...
[PRODUCER] Consumer return:  200 OK
[PRODUCER] Producing 3...
[CONSUMER] Consuming 3...
[PRODUCER] Consumer return:  200 OK
[PRODUCER] Producing 4...
[CONSUMER] Consuming 4...
[PRODUCER] Consumer return:  200 OK
[PRODUCER] Producing 5...
[CONSUMER] Consuming 5...
[PRODUCER] Consumer return:  200 OK


In [1]:
#Asyncio
import asyncio
import threading
@asyncio.coroutine
def hello():
    print("Hello world  (%s)" % threading.currentThread())
    r = yield from asyncio.sleep(1)
    print("Hello again  (%s)" % threading.currentThread())
# 获取EventLoop
loop = asyncio.get_event_loop()
tasks = [hello(),hello()]
loop.run_until_complete(asyncio.wait(tasks))
loop.close()

Hello world  (<_MainThread(MainThread, started 140735175024640)>)
Hello world  (<_MainThread(MainThread, started 140735175024640)>)
Hello again  (<_MainThread(MainThread, started 140735175024640)>)
Hello again  (<_MainThread(MainThread, started 140735175024640)>)


In [11]:
import asyncio

@asyncio.coroutine
def wget(host):
    print('wget %s...' % host)
    connect = asyncio.open_connection(host, 80)
    reader, writer = yield from connect
    header = 'GET / HTTP/1.0\r\nHost: %s\r\n\r\n' % host
    writer.write(header.encode('utf-8'))
    yield from writer.drain()
    while True:
        line = yield from reader.readline()
        if line == b'\r\n':
            break
        print('%s header > %s' % (host, line.decode('utf-8').rstrip()))
    # Ignore the body, close the socket
    writer.close()

loop = asyncio.get_event_loop()
tasks = [wget(host) for host in ['www.sina.com.cn', 'www.sohu.com', 'www.163.com']]
loop.run_until_complete(asyncio.wait(tasks))
# loop.close()

wget www.sina.com.cn...
wget www.sohu.com...
wget www.163.com...
www.sina.com.cn header > HTTP/1.1 200 OK
www.sina.com.cn header > Content-Type: text/html
www.sina.com.cn header > Vary: Accept-Encoding
www.sina.com.cn header > X-Powered-By: shci_v1.03
www.sina.com.cn header > Server: nginx
www.sina.com.cn header > Date: Wed, 23 Mar 2016 06:09:42 GMT
www.sina.com.cn header > Last-Modified: Wed, 23 Mar 2016 06:06:49 GMT
www.sina.com.cn header > Expires: Wed, 23 Mar 2016 06:10:42 GMT
www.sina.com.cn header > Cache-Control: max-age=60
www.sina.com.cn header > Age: 50
www.sina.com.cn header > Content-Length: 545416
www.sina.com.cn header > X-Cache: HIT from cernet194-220.sina.com.cn
www.sina.com.cn header > Connection: close
www.sohu.com header > HTTP/1.1 200 OK
www.sohu.com header > Content-Type: text/html
www.sohu.com header > Content-Length: 90884
www.sohu.com header > Connection: close
www.sohu.com header > Date: Wed, 23 Mar 2016 06:07:52 GMT
www.sohu.com header > Server: SWS
www.sohu.c

({<Task finished coro=<wget() done, defined at <ipython-input-11-d8a870461c3c>:3> result=None>,
  <Task finished coro=<wget() done, defined at <ipython-input-11-d8a870461c3c>:3> result=None>,
  <Task finished coro=<wget() done, defined at <ipython-input-11-d8a870461c3c>:3> result=None>},
 set())

In [9]:
def g(n):
    for i in range(n):
        yield from d(i)
def d(x):
    yield x**2
for x in g(5):
    print(x)

0
1
4
9
16


In [12]:
#python 3.5 引入的  替换@asyncio.coroutine的async    和 替换yiled from 的 await
async def hello():
    print("Hello Async")
    r = await asyncio.sleep(1)
    print("Hello Again")
loop = asyncio.get_event_loop()
tasks = [hello(),hello()]
loop.run_until_complete(asyncio.wait(tasks))
loop.close()

Hello Async
Hello Async
Hello Again
Hello Again


In [ ]:
#aiohttp
import asyncio
from aiohttp import web

async def index(request):
    await asyncio.sleep(5)
    return web.Response(body=b'<h1>Index<h1>')
async def hello(request):
    await asyncio.sleep(5)
    text = '<h1>hello,%s!</h1>' % request.match_info['name']
    return web.Response(body=text.encode('utf-8'))
async def init(loop):
    app = web.Application(loop = loop)
    app.router.add_route('GET','/',index)
    app.router.add_route('GET','/hello/{name}',hello)
    srv = await loop.create_server(app.make_handler(),'127.0.0.1',8000)
    print('Server started at http://127.0.0.1:8000...')
    return srv
loop = asyncio.get_event_loop()
loop.run_until_complete(init(loop))
loop.run_forever()

Server started at http://127.0.0.1:8000...
